In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f

spark = SparkSession.builder.appName("Process File")\
    .getOrCreate()

In [2]:
df_aug = spark.read\
    .format("com.databricks.spark.csv")\
    .option("header", "false") \
    .option("delimiter", " ") \
    .option("inferSchema", "true") \
    .load("access_log_Aug95")

In [3]:
df_jul = spark.read\
    .format("com.databricks.spark.csv")\
    .option("header", "false") \
    .option("delimiter", " ") \
    .option("inferSchema", "true") \
    .load("access_log_Jul95")

In [4]:
df = df_aug.union(df_jul)

In [5]:
df = df.select(df._c0.alias("host"), \
                  f.translate(f.concat(df._c3, f.lit(" "), df._c4), "$#,[]-", "XYZ").alias("Timestamp"), \
                  f.concat(f.lit('"'), df._c5, f.lit('"')).alias("request"), \
                  df._c6.alias("cod retorno http"), \
                  df._c7.alias("Total bytes retornados"))

In [6]:
df.show()

+--------------------+--------------------+--------------------+----------------+----------------------+
|                host|           Timestamp|             request|cod retorno http|Total bytes retornados|
+--------------------+--------------------+--------------------+----------------+----------------------+
|   in24.inetnebr.com|01/Aug/1995:00:00...|"GET /shuttle/mis...|             200|                  1839|
|     uplherc.upl.com|01/Aug/1995:00:00...|    "GET / HTTP/1.0"|             304|                     0|
|     uplherc.upl.com|01/Aug/1995:00:00...|"GET /images/kscl...|             304|                     0|
|     uplherc.upl.com|01/Aug/1995:00:00...|"GET /images/MOSA...|             304|                     0|
|     uplherc.upl.com|01/Aug/1995:00:00...|"GET /images/USA-...|             304|                     0|
|ix-esc-ca2-07.ix....|01/Aug/1995:00:00...|"GET /images/laun...|             200|                  1713|
|     uplherc.upl.com|01/Aug/1995:00:00...|"GET /images

## 1. Número de hosts únicos.

In [7]:
df.select(f.countDistinct("host")).show()

+--------------------+
|count(DISTINCT host)|
+--------------------+
|              137979|
+--------------------+



## 2. O total de erros 404

In [8]:
df.where(df['cod retorno http'] == "404").count()

20871

## 3. Os 5 URLs que mais causaram erro 404.

In [9]:
df.where(df['cod retorno http'] == "404") \
    .groupBy("host", "cod retorno http") \
    .count() \
    .orderBy("count", ascending=False) \
    .show(5)

+--------------------+----------------+-----+
|                host|cod retorno http|count|
+--------------------+----------------+-----+
|hoohoo.ncsa.uiuc.edu|             404|  251|
|piweba3y.prodigy.com|             404|  156|
|jbiagioni.npt.nuw...|             404|  132|
|piweba1y.prodigy.com|             404|  114|
|www-d4.proxy.aol.com|             404|   91|
+--------------------+----------------+-----+
only showing top 5 rows



## 4. Quantidade de erros 404 por dia.

In [10]:
df.withColumn("Dia", f.substring("Timestamp", 1, 11)) \
    .where(df['cod retorno http'] == "404") \
    .groupBy("cod retorno http", "Dia") \
    .count() \
    .show()

+----------------+-----------+-----+
|cod retorno http|        Dia|count|
+----------------+-----------+-----+
|             404|27/Aug/1995|  367|
|             404|03/Jul/1995|  470|
|             404|10/Aug/1995|  306|
|             404|13/Jul/1995|  531|
|             404|06/Aug/1995|  373|
|             404|05/Aug/1995|  236|
|             404|14/Jul/1995|  411|
|             404|03/Aug/1995|  303|
|             404|13/Aug/1995|  216|
|             404|20/Aug/1995|  312|
|             404|16/Jul/1995|  257|
|             404|16/Aug/1995|  259|
|             404|25/Aug/1995|  415|
|             404|20/Jul/1995|  428|
|             404|23/Jul/1995|  233|
|             404|07/Aug/1995|  537|
|             404|18/Aug/1995|  256|
|             404|06/Jul/1995|  640|
|             404|27/Jul/1995|  336|
|             404|19/Jul/1995|  638|
+----------------+-----------+-----+
only showing top 20 rows



## 5. O total de bytes retornados.

In [11]:
df.select(f.sum("Total bytes retornados")).show()

+---------------------------+
|sum(Total bytes retornados)|
+---------------------------+
|            6.5524319796E10|
+---------------------------+

